# Train/dev split

In [1]:
from sklearn.model_selection import train_test_split

In [2]:
texts, ids = [], []
with open('train_reviews.txt') as f:
    for line in f:
        text_id, text = line.rstrip('\r\n').split('\t')
        texts.append(text)
        ids.append(text_id)

In [3]:
train_texts, dev_texts, train_ids, dev_ids = train_test_split(texts, ids)

In [4]:
train_aspects, dev_aspects = [], []
with open('train_aspects.txt') as f:
    for line in f:
        line = line.rstrip('\r\n')
        text_id = line.split('\t')[0]
        if text_id in train_ids:
            train_aspects.append(line)
        if text_id in dev_ids:
            dev_aspects.append(line)

In [5]:
train_sentiment, dev_sentiment = [], []
with open('train_cats.txt') as f:
    for line in f:
        line = line.rstrip('\r\n')
        text_id = line.split('\t')[0]
        if text_id in train_ids:
            train_sentiment.append(line)
        if text_id in dev_ids:
            dev_sentiment.append(line)

In [6]:
with open('train_split_aspects.txt', 'w') as f:
    for l in train_aspects:
        print(l, file=f)
with open('dev_aspects.txt', 'w') as f:
    for l in dev_aspects:
        print(l, file=f)
with open('train_split_reviews.txt', 'w') as f:
    for i, l in zip(train_ids, train_texts):
        print(i, l, sep="\t", file=f)
with open('dev_reviews.txt', 'w') as f:
    for i, l in zip(dev_ids, dev_texts):
        print(i, l, sep="\t", file=f)
with open('train_split_cats.txt', 'w') as f:
    for l in train_sentiment:
        print(l, file=f)
with open('dev_cats.txt', 'w') as f:
    for l in dev_sentiment:
        print(l, file=f)

# Baseline 1,2: категория и тональность упоминаний

Выделяем только аспекты, встретившиеся в train'е, приписываем самую частотную категорию.

In [7]:
import pandas as pd

In [8]:
train_asp = pd.read_csv(
    'train_split_aspects.txt', 
    delimiter='\t', 
    names=['text_id', 'category', 'mention', 'start', 'end', 'sentiment']
)
train_texts = pd.read_csv('train_split_reviews.txt', delimiter='\t', names=['text_id','text'])

In [9]:
import stanza
stanza.download('ru')

2023-01-17 14:18:57 INFO: Downloading default packages for language: ru (Russian) ...
2023-01-17 14:18:58 INFO: File exists: /Users/ceo/stanza_resources/ru/default.zip
2023-01-17 14:19:01 INFO: Finished downloading models and saved to /Users/ceo/stanza_resources.


In [10]:
nlp = stanza.Pipeline('ru', processors='tokenize,lemma')

2023-01-17 14:19:01 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-01-17 14:19:01 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| lemma     | syntagrus |

2023-01-17 14:19:01 INFO: Use device: cpu
2023-01-17 14:19:01 INFO: Loading: tokenize
2023-01-17 14:19:01 INFO: Loading: lemma
2023-01-17 14:19:01 INFO: Done loading processors!


In [11]:
def normalize(text):
    doc = nlp(text)
    words = [word.lemma for sent in doc.sentences for word in sent.words]
    return words

In [12]:
from collections import defaultdict, Counter

In [14]:
train_asp['norm_mention'] = [tuple(normalize(m)) for m in train_asp['mention']]

In [15]:
train_asp

,text_id,category,mention,start,end,sentiment,norm_mention
0,3976,Whole,ресторане,71,80,neutral,"(ресторан,)"
1,3976,Whole,ресторанах,198,208,neutral,"(ресторан,)"
2,3976,Whole,ресторане,256,265,neutral,"(ресторан,)"
3,3976,Service,Столик бронировали,267,285,neutral,"(столик, бронировали)"
4,3976,Service,администратор,322,335,positive,"(администратор,)"
...,...,...,...,...,...,...,...
3494,16630,Service,обслуживание,85,97,positive,"(обслуживание,)"
3495,16630,Food,Еда,99,102,positive,"(еда,)"
3496,16630,Service,персоналу,244,253,positive,"(персонал,)"
3497,16630,Whole,ресторан,294,302,positive,"(ресторан,)"


Строим частотный словарь "токенизированное упоминание + категория"

Категория - аспектная категория или тональность

In [16]:
def get_mention_category(data, cat_type):
    mention_categories = data.value_counts(subset=['norm_mention', cat_type])
    mention_categories_dict = defaultdict(dict)
    for key, value in mention_categories.items():
        mention_categories_dict[key[0]][key[1]] = value
    return {k: Counter(v).most_common(1)[0][0] for k, v in mention_categories_dict.items()}

In [17]:
best_mention_cat = get_mention_category(train_asp, 'category')

In [18]:
best_mention_cat

{('ресторан',): 'Whole',
 ('обслуживание',): 'Service',
 ('интерьер',): 'Interior',
 ('официант',): 'Service',
 ('кухня',): 'Food',
 ('место',): 'Whole',
 ('заведение',): 'Whole',
 ('еда',): 'Food',
 ('блюдо',): 'Food',
 ('цена',): 'Price',
 ('персонал',): 'Service',
 ('порция',): 'Food',
 ('официантка',): 'Service',
 ('пиво',): 'Food',
 ('атмосфера',): 'Interior',
 ('меню',): 'Food',
 ('сервис',): 'Service',
 ('музыка',): 'Interior',
 ('встретить',): 'Service',
 ('обстановка',): 'Interior',
 ('десерт',): 'Food',
 ('администратор',): 'Service',
 ('горячий',): 'Food',
 ('зал',): 'Interior',
 ('девушка',): 'Service',
 ('ждать',): 'Service',
 ('салаты',): 'Food',
 ('кафе',): 'Whole',
 ('обслуживал',): 'Service',
 ('принести',): 'Service',
 ('поесть',): 'Food',
 ('мясо',): 'Food',
 ('вечер',): 'Whole',
 ('салат',): 'Food',
 ('вино',): 'Food',
 ('столик',): 'Interior',
 ('напиток',): 'Food',
 ('живой', 'музыка'): 'Interior',
 ('счет',): 'Price',
 ('кофе',): 'Food',
 ('заказ',): 'Service',
 

In [19]:
best_mention_sentiment = get_mention_category(train_asp, 'sentiment')

In [20]:
best_mention_sentiment

{('интерьер',): 'positive',
 ('обслуживание',): 'positive',
 ('ресторан',): 'positive',
 ('место',): 'positive',
 ('кухня',): 'positive',
 ('официант',): 'positive',
 ('заведение',): 'positive',
 ('персонал',): 'positive',
 ('еда',): 'positive',
 ('цена',): 'positive',
 ('блюдо',): 'positive',
 ('атмосфера',): 'positive',
 ('порция',): 'positive',
 ('пиво',): 'positive',
 ('музыка',): 'positive',
 ('обстановка',): 'positive',
 ('меню',): 'positive',
 ('официантка',): 'negative',
 ('встретить',): 'positive',
 ('сервис',): 'positive',
 ('администратор',): 'positive',
 ('девушка',): 'positive',
 ('зал',): 'positive',
 ('десерт',): 'positive',
 ('вечер',): 'positive',
 ('обслуживал',): 'positive',
 ('ждать',): 'negative',
 ('кафе',): 'positive',
 ('поесть',): 'positive',
 ('горячий',): 'neutral',
 ('напиток',): 'neutral',
 ('салаты',): 'positive',
 ('столик',): 'positive',
 ('живой', 'музыка'): 'positive',
 ('провести', 'время'): 'positive',
 ('мясо',): 'positive',
 ('стол',): 'positive',


In [21]:
dev_texts = pd.read_csv('dev_reviews.txt', delimiter='\t', names=['text_id', 'text'])

In [22]:
dev_texts

,text_id,text
0,18372,Ресторанчик очень милый.бывала там не раз!Гото...
1,12678,"Замечательное заведение, мы с мужем просто в в..."
2,785,Добрый день! Отмечали свадьбу 18 августа. В об...
3,5155,Я люблю такие заведения..очень уютно..отличная...
4,6376,Впервые посетила это замечательное место! С пе...
...,...,...
66,7824,"Праздновала в этом ресторане свой Др, праздник..."
67,6962,Очаровательная Виктория просила об отзыве и я ...
68,8555,Выбрали это место для начала празднования с ко...
69,37516,"Здравствуйте, Я отмечала свой день рожденья в ..."


Длины упоминаний аспектов в трейне:

In [23]:
Counter([len(x) for x in best_mention_sentiment.keys()])

Counter({1: 582,
         2: 312,
         3: 157,
         4: 63,
         7: 7,
         5: 31,
         6: 11,
         12: 1,
         9: 1,
         10: 1,
         8: 1})

Будем учитывать только упоминания длиной 1-5 токенов:

In [24]:
def label_texts(text, mentions, sentiments, max_len=5):
    tokenized = [word for sent in nlp(text).sentences for word in sent.words]
    text_end = len(tokenized)
    for i, token in enumerate(tokenized):
        for l in reversed(range(max_len)):
            if i + l > text_end:
                continue
            span = tokenized[i:i + l]
            key = tuple([t.lemma for t in span])
            if key in mentions:
                start, end = span[0].start_char, span[-1].end_char
                yield mentions[key], text[start:end], start, end, sentiments[key]
                break

Применяем частотные данные к текстам из dev:

In [25]:
with open('dev_pred_aspects.txt', 'w') as f:
    for text, idx in zip(dev_texts['text'], dev_texts['text_id']):
        for asp in label_texts(text, best_mention_cat, best_mention_sentiment):
            print(idx, *asp, sep="\t", file=f)

# Baseline 3

Посчитаем упоминания аспектов с предсказанной тональностью, припишем
- `absence` - если нет упоминаний данной категории
- `both` - если есть упоминания с разной тональностью
- `positive/neutral/negative` - если все упоминания одной тональности

In [26]:
CATEGORIES = ['Whole', 'Interior', 'Service', 'Food', 'Price']

In [27]:
def get_full_sentiment(text, mentions, sentiment, max_len=5):
    asp_counter = defaultdict(Counter)
    for asp in label_texts(text, best_mention_cat, best_mention_sentiment, max_len):
        category, *_, sentiment = asp
        asp_counter[category][sentiment] += 1
    for c in CATEGORIES:
        if not asp_counter[c]:
            s = 'absence'
        elif len(asp_counter[c]) == 1:
            s = asp_counter[c].most_common(1)[0][0]
        else:
            s = 'both'
        yield c, s

In [28]:
with open('dev_pred_cats.txt', 'w') as f:
    for text, idx in zip(dev_texts['text'], dev_texts['text_id']):
        for c, s in get_full_sentiment(text, best_mention_cat, best_mention_sentiment):
            print(idx, c, s, sep="\t", file=f)